# Data Cleaning

## Import Libraries and Load the data

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import re
from nltk.tokenize import RegexpTokenizer

plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
np.set_printoptions(precision=4)

In [28]:
df = pd.read_csv('../../../GA/Capstone/scrapes/scrape_30-Mar-22.csv')

In [29]:
df.head()

,date,user,title,comment,opinion,price
0,Today 22:43,Dillon2019,RE: Any use to us folks here?,This was on the BBC yesterday but I didnât c...,Strong Buy,13.1
1,Today 22:41,JiffyBag,RE: Any use to us folks here?,Russiaâs Economy in Freefallhttps://www.yout...,No Opinion,13.1
2,Today 22:15,JackoDLad,Any use to us folks here?,Found the following on another board (EUA) and...,No Opinion,13.1
3,Today 20:00,Tymers,RE: Naughty boys,"Speedy, like Hydro at some point will be corre...",Strong Buy,13.1
4,Today 19:52,Tymers,RE: Choices,Choices is correct! Yet all comments are ones ...,Strong Buy,13.1


## Data Cleaning

### Date Column

As the date column labels the current day (Today) and 5 previous days listed as the weekday only (the day of month is omitted), some pre-cleaning must be completed on csv scrapes that include the current day and 5 days prior to the current day, prior to concantenating to any other scrapes, to account for different scrape dates.

In [30]:
weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Today']

In [31]:
# view which days are in the format of Day, Time

for weekday in weekdays:
    print(weekday, len(df.loc[df['date'].str.contains(weekday)]))

Mon 118
Tue 0
Wed 184
Thu 88
Fri 125
Sat 45
Sun 23
Today 110


In [32]:
# to access the first date in the format of day, month, year, work out how many are in the other format

bad_format = 0
for weekday in weekdays:
    bad_format += len(df.loc[df['date'].str.contains(weekday)])

bad_format

693

In [33]:
# view the first date with the correct format
# note when indexing with .loc, as the first line is 0 we dont need to add 1 to bad_format to view the proceding date

pd.to_datetime(df.loc[bad_format, 'date']).strftime('%d %b %Y')

'22 Mar 2022'

In [35]:
# calculate what day today is

today = pd.to_datetime(df.loc[bad_format, 'date']) + timedelta(7)
today.strftime('%d %b %Y')

'29 Mar 2022'

In [37]:
# work out what the other bad format days correspond to
today_m1 = today - timedelta(1)
today_m2 = today - timedelta(2)
today_m3 = today - timedelta(3)
today_m4 = today - timedelta(4)
today_m5 = today - timedelta(5)
today_m6 = today - timedelta(6)

In [38]:
# creat dict to map datetime.dayofweek to the bad format given in the csv
week_dict = {0:'Mon',
             1:'Tue',
             2:'Wed',
             3:'Thu',
             4:'Fri',
             5:'Sat',
             6:'Sun'}

In [39]:
# modify the dates with the bad format to align with the rest of the dataframe

df.loc[df['date'].str.contains('Today') ,'date'] = today.strftime('%d %b %Y')
df.loc[df['date'].str.contains(week_dict[today_m1.dayofweek]) ,'date'] = today_m1.strftime('%d %b %Y')
df.loc[df['date'].str.contains(week_dict[today_m2.dayofweek]) ,'date'] = today_m2.strftime('%d %b %Y')
df.loc[df['date'].str.contains(week_dict[today_m3.dayofweek]) ,'date'] = today_m3.strftime('%d %b %Y')
df.loc[df['date'].str.contains(week_dict[today_m4.dayofweek]) ,'date'] = today_m4.strftime('%d %b %Y')
df.loc[df['date'].str.contains(week_dict[today_m5.dayofweek]) ,'date'] = today_m5.strftime('%d %b %Y')
df.loc[df['date'].str.contains(week_dict[today_m6.dayofweek]) ,'date'] = today_m6.strftime('%d %b %Y')

In [40]:
df.head()

,date,user,title,comment,opinion,price
0,29 Mar 2022,Dillon2019,RE: Any use to us folks here?,This was on the BBC yesterday but I didnât c...,Strong Buy,13.1
1,29 Mar 2022,JiffyBag,RE: Any use to us folks here?,Russiaâs Economy in Freefallhttps://www.yout...,No Opinion,13.1
2,29 Mar 2022,JackoDLad,Any use to us folks here?,Found the following on another board (EUA) and...,No Opinion,13.1
3,29 Mar 2022,Tymers,RE: Naughty boys,"Speedy, like Hydro at some point will be corre...",Strong Buy,13.1
4,29 Mar 2022,Tymers,RE: Choices,Choices is correct! Yet all comments are ones ...,Strong Buy,13.1


### Concatenating Scrapes

Optional Steps - due to the unreliable nature of scraping data from the website, it is likely that more than one scrape will be required to build the larger datasets. The steps below naturally would need to be adapted based on the scrapes being combined. The date column is formated as per the above for the previous 5 days to the scrape date, so these steps may also need to be taken on separate csvs prior to concatenation. Removing duplicates to account for overlaps will also be neccessary.

In [43]:
df = pd.concat([df2, df3, df1, df4], ignore_index=True)

In [44]:
df.shape

(133800, 6)

### comment field

In [45]:
df.comment[20]

'Could be vanguard Aus? Grown their position significantly and manage huge assets:The assets under management (AUM) of Vanguard Investments Australia Limited is $104.7 billion, all of which is managed in discretionary accounts'

Prior to any NLP work it will be neccessary to remove any bad text encoding information from the comments field. At this stage we wont be looking to tokenize the data, rather to remove special characters and numbers

In [46]:
# see if there are any null rows in the data set
df[df.isnull().sum(axis=1)>0]

,date,user,title,comment,opinion,price
94185,12 Sep '20,Dce2020,RE: My 2 Peneth,NaN,No Opinion,19.9
94186,12 Sep '20,Dce2020,RE: My 2 Peneth,NaN,No Opinion,19.9


In [47]:
# this function will be used to process the comments into a clean comment column
# clean comment columns will be more workable for NLP

def text_clean(text):
    # string text
    st = str(text)
    
    # make lowercase
    lc = st.lower()
    
    # remove any web addresses
    rem_url = re.sub(r'http\S+', '', lc)
    
    # remove any special characters
    rem_char = re.sub(r'[^a-z ]+', '', rem_url)
    
    # remove whitespace
    rem_ws = re.sub('\s\s+', ' ', rem_char)
    
    # tokenize words
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_ws)
    
    # remove any words with 2 characters or less
    filtered_words = [w for w in tokens if len(w) > 2]
    
    # return tokens rejoined into a string
    return ' '.join(filtered_words)

In [48]:
df['comment_clean'] = df['comment'].map(lambda x: text_clean(x))

In [137]:
df.head()

,date,user,title,comment,opinion,price,datetime,comment_clean
0,31 Jan 2022,JiffyBag,RE: SAS,"For me, it was just a fun exercise to do, and ...",No Opinion,13.8,2022-01-31,for was just fun exercise and will have see wh...
1,31 Jan 2022,JiffyBag,RE: SAS,"Hi Walkabout,I fully acknoweldege your rationa...",No Opinion,13.8,2022-01-31,walkabouti fully acknoweldege your rationale e...
2,31 Jan 2022,rosso123,Newcrest pulling a fast one,"NCM wants this 5% done and dusted asap, otherw...",No Opinion,13.8,2022-01-31,ncm wants this done and dusted asap otherwise ...
3,31 Jan 2022,JiffyBag,RE: Why would u short this,"Merc,It would be fantastic if someone could ac...",No Opinion,13.8,2022-01-31,mercit would fantastic someone could actually ...
4,31 Jan 2022,Philbrim,Ups n downs.,We go down 5% and most people are so volatile....,No Opinion,13.8,2022-01-31,down and most people are volatile and everyone...


In [49]:
# see if there are any null rows in the data set
df[df.isnull().sum(axis=1)>0]

,date,user,title,comment,opinion,price,comment_clean
94185,12 Sep '20,Dce2020,RE: My 2 Peneth,NaN,No Opinion,19.9,nan
94186,12 Sep '20,Dce2020,RE: My 2 Peneth,NaN,No Opinion,19.9,nan


In [50]:
# drop any nul rows if neccessary
df.dropna(inplace=True)

In [51]:
df.shape

(133798, 7)

In [52]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [53]:
df.shape

(132558, 7)

In [54]:
df.head()

,date,user,title,comment,opinion,price,comment_clean
0,7 Dec '21,hydrogen,RE: Hydro,Yes Sir.,No Opinion,14.05,yes sir
1,7 Dec '21,Sycho,Hydro,Lets keep your 12:35 post to ourselves and kee...,No Opinion,14.05,lets keep your post ourselves and keep realyou...
2,7 Dec '21,TomE,GGPSP,"So what happened to the ""Expect a sudden jump ...",No Opinion,14.05,what happened the expect sudden jump price its...
3,7 Dec '21,Sycho,RE: Personally in my opinion,"LOL, now that's more like it.",No Opinion,14.05,lol now thats more like
4,7 Dec '21,Shady69,RE: Personally in my opinion,Hydro it's so well put it could bring a tear t...,No Opinion,14.05,hydro its well put could bring tear glass eye ...


## Export to csv

In [55]:
# save to csv - note this is a local path

df.to_csv('../../../GA/Capstone/scrapes/large_scrape_29-Mar-22_.csv', index=False)

## NLP EDA

### Import NLP Packages

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.text import Text
import re
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, precision_score, classification_report


tokenizer = RegexpTokenizer(r'\w+')